In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/RL

/content/drive/MyDrive/RL


In [4]:
import numpy as np
import random
import time

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

import EnvironmentV2
from EnvironmentV2 import HackingEnv

In [5]:
SIZE = 1000
EDGE = 10

In [6]:
network = EnvironmentV2.gen_network(SIZE, EDGE)

In [7]:
env = HackingEnv(network, SIZE)

In [8]:
episodes = 20
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        random.seed(time)
        action = env.action_return()[env.action_space.sample()]
        nextState, reward, done, _ = env.step(action)
        score += reward
    print(f"Episode: {episode}, Score: {score}")

Episode: 1, Score: -6.5
Episode: 2, Score: -1.5
Episode: 3, Score: 1.0
Episode: 4, Score: 11.0
Episode: 5, Score: 11.0
Episode: 6, Score: 1.0
Episode: 7, Score: -4.0
Episode: 8, Score: 11.0
Episode: 9, Score: 1.0
Episode: 10, Score: -1.5
Episode: 11, Score: 8.5
Episode: 12, Score: -1.5
Episode: 13, Score: -6.5
Episode: 14, Score: -9.0
Episode: 15, Score: 1.0
Episode: 16, Score: 1.0
Episode: 17, Score: 6.0
Episode: 18, Score: -9.0
Episode: 19, Score: 1.0
Episode: 20, Score: 13.5


In [9]:
actions = env.action_space.n

In [10]:
def build_model(actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=(1, 4)))
    model.add(Dense(24, activation='relu'))
    model.add(Flatten())
    model.add(Dense(actions, activation='linear'))
    return model

In [11]:
model = build_model(actions)

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 24)             120       
_________________________________________________________________
dense_1 (Dense)              (None, 1, 24)             600       
_________________________________________________________________
flatten (Flatten)            (None, 24)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 100       
Total params: 820
Trainable params: 820
Non-trainable params: 0
_________________________________________________________________


In [13]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [14]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=60000, visualize=False, verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training_utils.py:1446: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if isinstance(sample_weight_mode, collections.Mapping):
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Training for 60000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 9:07 - reward: 1.5000

/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   30/10000 [..............................] - ETA: 3:29 - reward: 1.5000

/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) instead
  batch_idxs

10000/10000 [==============================] - 83s 8ms/step - reward: 1.4585
416 episodes - episode_reward: 35.002 [26.000, 36.000] - loss: 16.943 - mae: 19.918 - mean_q: 27.754

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 84s 8ms/step - reward: 1.4487
417 episodes - episode_reward: 34.771 [26.000, 36.000] - loss: 21.661 - mae: 24.351 - mean_q: 33.253

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 84s 8ms/step - reward: 1.4485
417 episodes - episode_reward: 34.765 [21.000, 36.000] - loss: 20.968 - mae: 23.793 - mean_q: 32.436

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 86s 9ms/step - reward: 1.4460
416 episodes - episode_reward: 34.702 [26.000, 36.000] - loss: 19.791 - mae: 23.268 - mean_q: 31.614

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 90s 9ms/step - reward: 1.4385
417 episodes - episode_reward: 34.525 [26.000, 36.000] - loss: 19.3

In [15]:
results = dqn.test(env, nb_episodes=150, visualize=False)
print(np.mean(results.history['episode_reward']))

Testing for 150 episodes ...
Episode 1: reward: 36.000, steps: 24
Episode 2: reward: 36.000, steps: 24
Episode 3: reward: 36.000, steps: 24
Episode 4: reward: 36.000, steps: 24
Episode 5: reward: 36.000, steps: 24
Episode 6: reward: 36.000, steps: 24
Episode 7: reward: 36.000, steps: 24
Episode 8: reward: 36.000, steps: 24
Episode 9: reward: 36.000, steps: 24
Episode 10: reward: 36.000, steps: 24
Episode 11: reward: 36.000, steps: 24
Episode 12: reward: 36.000, steps: 24
Episode 13: reward: 36.000, steps: 24
Episode 14: reward: 36.000, steps: 24
Episode 15: reward: 36.000, steps: 24
Episode 16: reward: 36.000, steps: 24
Episode 17: reward: 36.000, steps: 24
Episode 18: reward: 36.000, steps: 24
Episode 19: reward: 36.000, steps: 24
Episode 20: reward: 36.000, steps: 24
Episode 21: reward: 36.000, steps: 24
Episode 22: reward: 36.000, steps: 24
Episode 23: reward: 36.000, steps: 24
Episode 24: reward: 36.000, steps: 24
Episode 25: reward: 36.000, steps: 24
Episode 26: reward: 36.000, st